In [1]:
import numpy as np
import cvxpy as cp
import json 
import json 
import pandas as pd 
import polars as pl 
import os 
from lower_mpc_old import step_lower_mpc, LMPCDataBuffer
from upper_mpc import step_upper_level, UMPCDataBuffer
from tqdm import tqdm 
import time 
from datetime import timedelta
import matplotlib.pyplot as plt 
%load_ext autoreload
%autoreload 2

In [2]:
root_folder = "/home/alqua/papers/mbs_aggregation_paper_code/mpc/"
params_path = "sys_id/results/"

In [3]:
data_path_wsl = "/home/alqua/data/pump_station_data/"
upper_mpc_data = pl.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))
upper_mpc_data_pan = pl.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))

In [4]:
start_date = pd.to_datetime("2024-02-15 00:00:00+00:00")
end_date = pd.to_datetime("2024-02-25 08:00:00+00:00")

upper_mpc_data = upper_mpc_data.filter((pl.col("time") >= start_date) & 
                                       (pl.col("time") <= end_date)
                                       ).with_columns(pl.col("CO2Emission")/1000, pl.col("price")/1000)

upper_mpc_data = upper_mpc_data.with_columns(pl.col("time").dt.day().alias("day_index"))
day_sliced_df = upper_mpc_data.filter(pl.col("day_index") == 15)

In [5]:
with open('lower_mpc_coefficients.json', 'r') as json_file:
    models_coefficients = json.load(json_file)

In [6]:
lmpc_data = LMPCDataBuffer()
umpc_data = UMPCDataBuffer()

In [7]:
lmpc_data.initialize()

In [8]:
slice_df = upper_mpc_data.select(pl.col(["time", "inflow_kf", "CO2Emission","price"]))
inflow_kf = slice_df["time", "inflow_kf"].upsample(every="1m", time_column= "time").fill_null(strategy="forward")["inflow_kf"]

k = 0 
zs = 3 
N = 60
horizon = 24
step_size = 1
max_start = 1 #len(slice_df) - horizon + 1

for start_index in tqdm(range(0, max_start, step_size)):
    prices_values = slice_df["price"].slice(start_index, horizon)
    co2_values = slice_df["CO2Emission"].slice(start_index, horizon)
    inflow_values = slice_df["inflow_kf"].slice(start_index, horizon)

    start_time_umpc =  time.time()
    umpc_opt_results = step_upper_level(
        horizon=horizon,
        prices_values=prices_values,
        co2_progn_values=co2_values,
        inflow_values=inflow_values,
        h_init=lmpc_data.data["height_sys"][-1],
        energy_init=lmpc_data.data["p1_power"][-1] + lmpc_data.data["p3_power"][-1] + lmpc_data.data["p4_power"][-1],
        Qout_init=lmpc_data.data["qout"][-1])

    end_time_umpc = time.time() - start_time_umpc
    umpc_opt_results["opt_time_umpc"] = end_time_umpc
    umpc_data.update(umpc_opt_results)
    #print(start_index, start_index+horizon)

    inflow_kf = np.ones(600)*slice_df["inflow_kf"][start_index]
    
    for k in range(0,60):

        u_stack = np.vstack([lmpc_data.data["u1"][-zs:],
                            lmpc_data.data["u2"][-zs:], 
                            lmpc_data.data["u3"][-zs:]])

        power_stack = np.vstack([lmpc_data.data["p1_power"][-zs:],
                            lmpc_data.data["p3_power"][-zs:], 
                            lmpc_data.data["p4_power"][-zs:]])

        start_time_lmpc =  time.time()
        res_dict  = step_lower_mpc(Qin_est = inflow_kf[k+zs:k+zs+N+zs],
                                Qout_meas = lmpc_data.data["qout"][-zs:],
                                h_meas = lmpc_data.data["height_sys"][-zs:],
                                w_meas = u_stack,
                                E_meas = power_stack,
                                P_meas = lmpc_data.data["pressure_sys"][-zs:],
                                h_ref = umpc_opt_results["height"],
                                trigger = [1,0,0],
                                N = 60,
                                zs = zs,
                                models_coefficients = models_coefficients)
        end_time_lmpc = time.time() - start_time_lmpc
        res_dict["opt_time_lmpc"] = end_time_lmpc

        res_dict['time'] = slice_df["time"][start_index] + timedelta(minutes=k)
        res_dict['height_ref'] = umpc_opt_results["height"]
        res_dict['qin'] = inflow_kf[k+zs:k+zs+N+zs]
        #res_dict['time'] = 0 
        
        lmpc_data.update(res_dict)



  0%|          | 0/1 [00:00<?, ?it/s]


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



100%|██████████| 1/1 [00:17<00:00, 17.59s/it]


In [9]:
opt_umpc = umpc_data.to_dataframe()
opt_umpc

Qout,height,co2_progn,cum_energy,da_price,objective,opt_time_umpc
f64,f64,f64,f64,f64,f64,f64
592.241879,199.999998,1.013,31.673487,0.06113,1569.836879,0.45914


In [10]:
opt_lmpc = lmpc_data.to_dataframe()
opt_lmpc

time,u1,u2,u3,p1_power,p3_power,p4_power,qout,qin,height_sys,height_ref,pressure_sys,opt_time_lmpc
"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,"array[f64, 63]",f64,f64,f64,f64
2024-02-15 00:00:00 UTC,816.0,3.0,5.0,33.2,-0.0,0.0,0.0,"[644.241879, 644.241879, … 644.241879]",111.0,199.999998,0.6,0.298916
2024-02-15 00:01:00 UTC,831.0,5.0,9.0,35.9,0.1,0.1,496.9,"[644.241879, 644.241879, … 644.241879]",151.3,199.999998,0.5,0.310108
2024-02-15 00:02:00 UTC,846.0,7.0,13.0,38.1,0.2,0.3,616.2,"[644.241879, 644.241879, … 644.241879]",160.5,199.999998,0.6,0.319101
2024-02-15 00:03:00 UTC,860.0,9.0,16.0,39.3,0.3,0.4,620.9,"[644.241879, 644.241879, … 644.241879]",162.3,199.999998,0.6,0.302198
2024-02-15 00:04:00 UTC,874.0,10.0,19.0,40.2,0.4,0.6,624.1,"[644.241879, 644.241879, … 644.241879]",163.8,199.999998,0.6,0.310704
…,…,…,…,…,…,…,…,…,…,…,…,…
2024-02-15 00:55:00 UTC,1256.0,0.0,0.0,59.1,-0.0,0.0,639.2,"[644.241879, 644.241879, … 644.241879]",192.8,199.999998,0.6,0.301507
2024-02-15 00:56:00 UTC,1258.0,0.0,0.0,59.2,0.0,0.0,639.6,"[644.241879, 644.241879, … 644.241879]",193.1,199.999998,0.6,0.311072
2024-02-15 00:57:00 UTC,1260.0,0.0,0.0,59.3,0.0,0.0,640.7,"[644.241879, 644.241879, … 644.241879]",193.4,199.999998,0.6,0.304238
